In [2]:
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)
from glob import glob
import json
from pprint import pprint
import numpy as np
import os
from pathlib import Path
from sklearn.tree import DecisionTreeClassifier
import sklearn
import pickle
import sys

In [4]:
test_dataset = sys.argv[1]
def get_all_dataset_files(path):
    return [os.path.join(dp, f) for dp, dn, filenames in os.walk(path) for f in filenames if os.path.splitext(f)[1] in ['.txt','.json'] ]

In [5]:
test_files = get_all_dataset_files(test_dataset)

In [9]:
hashes = {}
for test_file in test_files:
    hash_val = Path(test_file).parts[-1].replace(".json","") if os.path.splitext(test_file)[1] == ".json" else Path(test_file).parts[-2]
    file_paths = hashes.get(hash_val, {})
    p = Path(test_file).parts
    if "Malware" in test_file:
        file_paths["category"] = "Malware"
        file_paths["sub_category"] = p[-3]
    elif "Benign" in test_file: 
        file_paths["category"] = "Benign"
        file_paths["sub_category"] = "Benign"

    if os.path.splitext(test_file)[1] == ".json":
        file_paths["dynamic_analysis"] = test_file
    elif p[-1] == "String.txt":
        file_paths["static_analysis_strings"] = test_file
    elif p[-1] == "Structure_Info.txt":
        file_paths["static_analysis_structure"] = test_file
    hashes[hash_val] = file_paths
    
rows = []
for hash_val in hashes:
    row = {"hash":hash_val}
    row.update(hashes[hash_val])
    rows.append(row)
    
df = pd.DataFrame(rows)
print(df.columns)

Index([], dtype='object')


In [10]:
# Feature extraction
def check_string_presence(file_path, string=""):
    if os.path.isfile(file_path):
        if string in open(file_path).read():
            return 1
        else:
            return 0
    return 0
check_string_presence = np.vectorize(check_string_presence)

df["virus_total"] = check_string_presence(df["dynamic_analysis"],string="VirusTotal")
df["icmp_traffic_presence"] = check_string_presence(df["dynamic_analysis"],string="Generates some ICMP traffic")
df["high_port_presence"] = check_string_presence(df["dynamic_analysis"],string="Communication to multiple IPs on high port numbers")
df["shit_over_string_presence"] = check_string_presence(df["static_analysis_strings"],string="Shit,IsOver!!") # For worms

KeyError: 'dynamic_analysis'

In [ ]:
feature_cols = ["virus_total","icmp_traffic_presence","high_port_presence","shit_over_string_presence"]
inv_category_mappings = {1:"Malware",0:"Benign"}
clf = pickle.load("malware_detection.model")

df["predicted"] = clf.predict(df[feature_cols])
df["predicted"] = df["predicted"].map(inv_category_mappings)
df[["hash","predicted"]].to_csv("output.csv",index=False)
